# --- Day 4: Giant Squid ---
## Part 1
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1
```

```
22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19
```
```
 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6
```
```
14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```
After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
Finally, 24 is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?


In [1]:
import numpy as np
from typing import List, Set, Tuple

In [2]:
with open("../data/day04.dat", "r") as IH:
    drawn_numbers = [int(x) for x in next(IH).strip().split(",")]
    
    built_boards: np.array = np.zeros((5, 5)).reshape((1, 5, 5))
    current_board: List[str] = []
    
    for line in IH:
        if line.strip() == "" and len(current_board) != 0:
            built_boards = np.append(built_boards, np.array(current_board).astype("i").reshape((1, 5, 5)), axis=0)
            current_board = []
        current_board.extend(x for x in line.strip().split(" ") if x != "")
    
    built_boards = np.append(built_boards, np.array(current_board).astype("i").reshape((1, 5, 5)), axis=0)

built_boards = built_boards[1:]

In [3]:
def find_winner(drawn_numbers: List[int], built_boards: np.array) -> Tuple[Tuple[np.array, np.array], int]:
    matched_board_numbers = np.zeros(built_boards.shape).astype("bool")
    
    for n in drawn_numbers:
        matched_board_numbers = (matched_board_numbers + (built_boards == n)).astype("bool")
        winning_board_id_list = set()
        
        for axis in range(1, 3):
            completed_line = (matched_board_numbers.sum(axis) == 5).sum(1) > 0
            if any(completed_line):
                winning_board_id_list = set.union(winning_board_id_list, set(np.where(completed_line == 1)[0].tolist()))

        for winning_board_id in list(winning_board_id_list):
            return built_boards[winning_board_id, :, :], matched_board_numbers[winning_board_id, :, :], n, winning_board_id        

In [4]:
board, matched, n, winning_id = find_winner(drawn_numbers, built_boards)

In [5]:
board[np.logical_not(matched)].sum().astype("int") * n

8442

## --- Part Two ---

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?


In [6]:
import copy as cp

In [7]:
def find_last_winner(drawn_numbers: List[int], built_boards: np.array) -> List[int]:
    matched_board_numbers = np.zeros(built_boards.shape).astype("bool")
    has_won = set()
    winning_boards_order = []
    
    for n in drawn_numbers:
        matched_board_numbers = (matched_board_numbers + (built_boards == n)).astype("bool")
        winning_board_id_list = set()
        
        for axis in range(1, 3):
            completed_line = (matched_board_numbers.sum(axis) == 5).sum(1) > 0
            if any(completed_line):
                winning_board_id_list = set.union(winning_board_id_list, set(np.where(completed_line == 1)[0].tolist()))

        for winning_board_id in list(winning_board_id_list):
            if not winning_board_id in has_won:
                winning_boards_order.append((built_boards[winning_board_id, :, :], cp.copy(matched_board_numbers[winning_board_id, :, :]), n, winning_board_id))
            has_won.add(winning_board_id)
            matched_board_numbers[winning_board_id, :, :] = np.zeros((5, 5)).astype("bool")

    return winning_boards_order

In [8]:
board, matched, n, winning_id = find_last_winner(drawn_numbers, built_boards)[-1]

In [9]:
board[np.logical_not(matched)].sum().astype("int")*n

4590